<a href="https://colab.research.google.com/github/DonnaVakalis/Personal/blob/master/DJW_slack_word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For the code in this notebook, I am indebted to:


*   https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/
*   https://stackoverflow.com/questions/56866697/python-read-json-files-from-all-sub-directories



 

In [0]:
# IMPORTS

import os
import glob
import pandas as pd
from pathlib import Path

In [3]:
# MOUNT GOOGLE DRIVE

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# LOCATE RAW DATA FILES
os.chdir("/content/gdrive/My Drive/RawDat/Slack_export_June8")
working_directory = os.getcwd()
 
sub_directories = [working_directory + "/" + x for x in os.listdir(working_directory) if os.path.isdir(working_directory + "/"+x)]
all_json_files = []

for sub_dir in sub_directories:
    os.chdir(sub_dir)
    for file in glob.glob("*.json"):
        all_json_files.append(sub_dir + "/" + file)

# Get back to original working directory
os.chdir(working_directory)

In [5]:
# temp check
#print(all_json_files)
len(all_json_files) #2457

2457

In [6]:
# LOAD ALL JSON FILES INTO ONE LIST 
data_container = []
counter_totals = 0
counter_failures = 0
for x in all_json_files:
      counter_totals += 1
      try: 
          df = pd.read_json(x)
          data_container.append(df)
      except (ValueError):
          counter_failures += 1
          continue
      
print(counter_totals) # 2457
print(counter_failures) # 2

2457
2


In [0]:
# MAKE THE LIST INTO A DATAFRAME
all_data = pd.concat(data_container)

In [0]:
for col in all_data.columns: 
    print(col)

all_data.info

In [0]:
# try writing to csv

import csv
export_csv = all_data.to_csv (r'/content/gdrive/My Drive/RawDat/pandaresult.csv', index = None, header=True) # here you have to write path, where result file will be stored
 
# all_data.to_csv('slack_dataframe.csv’) # relative position
# gives syntax error: |"EOL while scanning string literal"

In [0]:
files

<generator object Path.glob at 0x7f7dd75de938>

In [0]:
# Get data

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving train_E6oV3lV.csv to train_E6oV3lV (1).csv
User uploaded file "train_E6oV3lV.csv" with length 3103165 bytes
